In [1]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(1249583)

# Data Preparation

<img src='lesson13_data.png'>

In [2]:
def str2ascii_arr(name):
    """
    0-255
    """
    arr = [ord(c) for c in name]
    return arr, len(arr)

In [3]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size=256, hidden_size=256, output_size=18, n_layers=1):
        """
        Because word embedding is working with ascii. It has to use `input_size=256, hidden_size=256`
        """
        super().__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        # input_size 256, hidden_size 256.
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, input):
        # Sung Kim run this all at once (over the whole input sequence)
        # input = B x S . size(0) = B
        batch_size = input.size(0)
        
        # input: B x S -- (transpose) --> S x B
        input = input.t()
        
        # Embedding S x B -> S x B x I (embedding size)
        print(f" input size: {input.size()}")
        embedded = self.embedding(input)
        embedded = embedded.clone().detach() # Make new tensor because of `EmbeddingGrad`
        print(f" embeddding size: {embedded.size()}")
        
        # Make a hidden
        hidden = self._init_hidden(batch_size)
        output, hidden = self.gru(embedded, hidden)
        print(f" gru hidden output: {hidden.size()}")
        
        # Use last layer output as FC's input
        # No need to unpack, since we are going to use hidden
        fc_output = self.fc(hidden)
        print(f" fc output: {fc_output.size()}")
        return fc_output
        
    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size)
        return torch.tensor(hidden, dtype=torch.float)

In [88]:
# in torch.Size([1, 6]) 'adylov'
# out torch.Size([1, 1, 18]) 18 countries

# Zero padding

<img src='zero_padding.png'>

In [89]:
def pad_sequences(vectorized_seqs, seq_lengths):
    seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max()), dtype=torch.long)
    for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seq_len] = torch.tensor(seq, dtype=torch.long)
    return seq_tensor

In [90]:
def make_variables(names):
    sequence_and_length = [str2ascii_arr(name) for name in names]
    vectorized_seqs = [sl[0] for sl in sequence_and_length]
    seq_lengths = torch.tensor([sl[1] for sl in sequence_and_length])
    return pad_sequences(vectorized_seqs, seq_lengths)

In [91]:
make_variables(['az', 'ab'])

tensor([[ 97, 122],
        [ 97,  98]])

In [97]:
classifier = RNNClassifier()
arr, _ = str2ascii_arr('adylov')
inp = torch.tensor([arr], dtype=torch.long)
out = classifier(inp)
print(f"\nin: {inp.size()}, \nout: {out.size()}")

 input size: torch.Size([6, 1])
 embeddding size: torch.Size([6, 1, 256])
 gru hidden output: torch.Size([1, 1, 256])
 fc output: torch.Size([1, 1, 18])

in: torch.Size([1, 6]), 
out: torch.Size([1, 1, 18])


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [100]:
names = ['adylov', 'solan', 'hard', 'san']
classifier = RNNClassifier()
inputs = make_variables(names)
out = classifier(inputs)
print(f"\nbatch in: {inputs.size()}, \nbatch out: {out.size()}")

 input size: torch.Size([6, 4])
 embeddding size: torch.Size([6, 4, 256])
 gru hidden output: torch.Size([1, 4, 256])
 fc output: torch.Size([1, 4, 18])

batch in: torch.Size([4, 6]), 
batch out: torch.Size([1, 4, 18])


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


# Utilities

In [191]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    print(cm)
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    y_test = []
    y_pred = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            y_test.append(int(target))
            y_pred.append(int(pred))
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    # Confusion matrix
    confusion_mtx = confusion_matrix(y_test, y_pred)
    plot_confusion_matrix(confusion_mtx, classes=[i for i in range(1, 10 + 1)], normalize=True,
                          title='Normalized confusion matrix')


# Dataset

In [169]:
trainset = pd.read_csv('names_train.csv', header=None)
testset = pd.read_csv('names_test.csv', header=None) 

In [170]:
headers = ['name', 'country']
trainset.columns = headers
testset.columns = headers

In [171]:
countries = list(trainset.country.drop_duplicates())
countries

['Czech',
 'German',
 'Arabic',
 'Japanese',
 'Chinese',
 'Vietnamese',
 'Russian',
 'French',
 'Irish',
 'English',
 'Spanish',
 'Greek',
 'Italian',
 'Portuguese',
 'Scottish',
 'Dutch',
 'Korean',
 'Polish']

In [130]:
# Majority of dataset is `Russian`
trainset.country.value_counts()

Russian       6272
English       2445
Arabic        1333
Japanese       660
German         482
Italian        472
Czech          346
Spanish        198
Dutch          198
French         184
Chinese        178
Irish          154
Greek          135
Polish          92
Scottish        66
Korean          62
Portuguese      49
Vietnamese      48
Name: country, dtype: int64

In [131]:
# So as trainset
testset.country.value_counts()

Russian       3136
English       1223
Arabic         667
Japanese       331
German         242
Italian        237
Czech          173
Spanish        100
Dutch           99
French          93
Chinese         90
Irish           78
Greek           68
Polish          47
Scottish        34
Korean          32
Vietnamese      25
Portuguese      25
Name: country, dtype: int64

In [154]:
trainset.iloc[0]['country']

'Czech'

In [165]:
class NameDataSet(Dataset):
    def __init__(self, filename='names_train.csv'):
        trainset = pd.read_csv('names_train.csv', header=None)
        trainset.columns = ['name', 'country']
        countries = list(trainset.country.drop_duplicates())

        self.trainset = trainset
        self.countries = countries
        self.len = len(trainset)        

    def __getitem__(self, index):
        country = self.trainset.iloc[index]['country']
        return self.trainset.iloc[index]['name'], self.countries.index(country)

    def __len__(self):
        return self.len
        

In [172]:
train_dataset = NameDataSet()
test_dataset = NameDataSet('names_test.csv')

In [177]:
train_dataset.countries.index('Czech')

0

In [185]:
train_loader = DataLoader(dataset=train_dataset, batch_size=2, num_workers=2, shuffle=True) # 2 * 9 * 743 
test_loader = DataLoader(dataset=test_dataset, batch_size=2, num_workers=2) # 4 * 25 * 67

In [186]:
for i, (inputs, labels) in enumerate(train_loader):
    print(inputs, labels)

('Jvanetsky', 'Grigorishin') tensor([6, 6])
('Ingram', 'Lapegin') tensor([9, 6])
('Frederikson', 'Vasyutinsky') tensor([9, 6])
('Sciacca', 'Portnov') tensor([12,  6])
('Antyushin', 'Rustici') tensor([ 6, 12])
('Shamgulov', 'Sarkozi') tensor([6, 7])
('Saito', 'Por') tensor([3, 6])
('Abzgildin', 'Hawkridge') tensor([6, 9])
('Gonzalez', 'Abadi') tensor([10,  2])
('Belosohov', 'Opp') tensor([6, 0])
('Gros', 'Tyrrell') tensor([1, 9])
('Balanchivadze', 'Jatkov') tensor([6, 6])
('Maruyama', 'Oakes') tensor([3, 9])
('Zhelekhovsky', 'Ufimov') tensor([6, 6])
('Janovitsky', 'Altoviti') tensor([ 6, 12])
('Farnum', 'Tuma') tensor([9, 2])
('Curran', 'Lokhov') tensor([8, 6])
('Uss', 'Chukhnovsky') tensor([6, 6])
('Upjohn', 'Haik') tensor([9, 2])
('Southgate', 'Yanishin') tensor([9, 6])
('Takagi', 'Mitchell') tensor([3, 9])
('Kenning', 'Renskov') tensor([9, 6])
('Hasbulatov', 'Makharov') tensor([6, 6])
('Noble', 'Avdonin') tensor([9, 6])
('Maurice', 'Kalakutsky') tensor([8, 6])
('Agol', 'Overing') ten

('Maloof', 'Lihtentul') tensor([2, 6])
('Awanesyan', 'Batsevich') tensor([6, 6])
('Glazkov', 'Khalil') tensor([6, 9])
('Woo', 'Sowka') tensor([ 4, 17])
('Blundell', 'Vesich') tensor([9, 6])
('Djunusov', 'Ronald') tensor([6, 9])
('Tomashpolsky', 'Saliba') tensor([6, 2])
('Vistitsky', 'Abyshev') tensor([6, 6])
('Mojeiko', 'Jamschikov') tensor([6, 6])
('Matzukevich', 'Jumonji') tensor([6, 3])
('Vavra', 'Durand') tensor([6, 7])
('Gluhov', 'Villalobos') tensor([ 6, 10])
('Walentowicz', 'So') tensor([17, 16])
('Duck', 'Nazari') tensor([9, 2])
('Lutoshnikov', 'Makushev') tensor([6, 6])
('Atalikov', 'Ohira') tensor([6, 3])
('Leedham', 'Otrohov') tensor([9, 6])
('Fairweather', 'Atroshenko') tensor([9, 6])
('Birentsveig', 'Grigolyuk') tensor([6, 6])
('Kalihov', 'Mandel') tensor([6, 1])
('Ghannam', 'Abboud') tensor([2, 2])
('Ichiyusai', 'Zheltoukhov') tensor([3, 6])
('Ataev', 'Hinrichs') tensor([6, 1])
('Simoes', 'Haik') tensor([13,  2])
('Knigge', 'Sapienti') tensor([ 9, 12])
('Bonfils', 'Harnes

('Stringer', 'Fossard') tensor([9, 9])
('Khouri', 'Rong') tensor([2, 4])
('Asanuma', 'Abrasimov') tensor([3, 6])
('Abrakhimoff', 'Iskortsev') tensor([6, 6])
('Schmitt', 'Baklashov') tensor([1, 6])
('Nakhutin', 'Mochanov') tensor([6, 6])
('Leggett', 'Gorchakovsky') tensor([9, 6])
('Stacey', 'Weber') tensor([9, 1])
('Tahan', 'Cham') tensor([2, 2])
('Jamov', 'Balakirev') tensor([6, 6])
('Farnsworth', 'Yakimychev') tensor([9, 6])
('Owen', 'Klossner') tensor([9, 1])
('Takagaki', 'Humphries') tensor([3, 9])
('Tzenin', 'Vysochin') tensor([6, 6])
('Timakov', 'Danyushevsky') tensor([6, 6])
('Zoucha', 'Hammond') tensor([0, 9])
('Panek', 'Bassi') tensor([ 0, 12])
('Willis', 'Agababyan') tensor([9, 6])
('Vanyashin', 'Sakiev') tensor([6, 6])
('Vinidiktov', 'Waghorn') tensor([6, 9])
('Mair', 'Zimonin') tensor([9, 6])
('Shalyugin', 'Tahan') tensor([6, 2])
('Shaihutdinov', 'Yeardley') tensor([6, 9])
('Dertilis', 'Andel') tensor([11, 15])
('Kill', 'Thao') tensor([9, 5])
('Donnchadh', 'Felton') tensor([

('Kaletkin', 'Sitz') tensor([6, 1])
('Awinoff', 'Hodowal') tensor([6, 0])
('Yuhno', 'Leonard') tensor([6, 9])
('Geisler', 'Hitarov') tensor([1, 6])
('Gudkov', 'Steadman') tensor([6, 9])
('Abakeliya', 'Chertorinsky') tensor([6, 6])
('Nahas', 'Jagubsky') tensor([2, 6])
('Tholberg', 'Daher') tensor([15,  2])
('Adamovitch', 'Gutermuth') tensor([6, 1])
('Handal', 'Chugaev') tensor([2, 6])
('Laurito', 'Nasyrov') tensor([12,  6])
('Hole', 'Kalinchuk') tensor([9, 6])
('Ling', 'Moghadam') tensor([9, 2])
('Jatzuba', 'Murogov') tensor([6, 6])
('Agulnik', 'Olenev') tensor([6, 6])
('Mansour', 'Nussbaum') tensor([2, 1])
('Mcdonald', 'Gomes') tensor([14, 13])
('Lefurgey', 'Awad') tensor([7, 2])
('Jakhaev', 'Olguin') tensor([ 6, 10])
('Uchuev', 'Shirnin') tensor([6, 6])
('Adachi', 'Sheehan') tensor([3, 9])
('Fazleev', 'Hovanec') tensor([6, 0])
('Zdunowski', 'Batten') tensor([17,  9])
('Penev', 'Dobreitser') tensor([6, 6])
('Tsehansky', 'Keeble') tensor([6, 9])
('Zavodskoi', 'Losik') tensor([6, 6])
('J

('Porkhunov', 'Gillett') tensor([6, 9])
('Gaubrich', 'Ebert') tensor([6, 6])
('Shalyapin', 'Durkin') tensor([6, 6])
('Lebedinets', 'Wirnhier') tensor([6, 1])
('Lovell', 'Mullen') tensor([9, 8])
('Scavo', 'Neverov') tensor([12,  6])
('Isa', 'Haddad') tensor([2, 2])
('Deryagin', 'Miyajima') tensor([6, 3])
('Saker', 'Wizner') tensor([6, 0])
('Entwistle', 'Murase') tensor([9, 3])
('Shamon', 'Rahmaninov') tensor([2, 6])
('Ichigawa', 'Hamnett') tensor([3, 9])
('Shahnovsky', 'Lygin') tensor([6, 6])
('Kennett', 'Mcguire') tensor([9, 8])
('Awakyan', 'Sabbag') tensor([6, 2])
('Usynin', 'Lyjin') tensor([6, 6])
('Pechuev', 'Davydenkov') tensor([6, 6])
('Fionn', 'Tsaliev') tensor([8, 6])
('Mihalitsin', 'Agurski') tensor([6, 6])
('Belgibaev', 'Malec') tensor([6, 0])
('Valiullin', 'Redin') tensor([6, 9])
('Engel', 'Kabirov') tensor([6, 6])
('Thomson', 'Barnard') tensor([9, 9])
('BestujevRyumin', 'Tucker') tensor([6, 9])
('Kaglantge', 'Anistratenko') tensor([11,  6])
('Ba', 'Lichintser') tensor([2, 6]

...

('Tyne', 'Pears') tensor([9, 9])
('Sneiders', 'Fothergill') tensor([15,  9])
('Demall', 'Klerx') tensor([ 0, 15])
('Kuang', 'Ponizovsky') tensor([4, 6])
('Minin', 'Okubo') tensor([6, 3])
('Kingscott', 'Kan') tensor([9, 4])
('Dublyansky', 'Mikhail') tensor([6, 2])
('Connor', 'Imamura') tensor([9, 3])
('Antar', 'Toland') tensor([2, 9])
('Zheltukhin', 'Shamanin') tensor([6, 6])
('Garofalo', 'Nassar') tensor([12,  2])
('Nagai', 'Paton') tensor([3, 9])
('Durnovo', 'Livson') tensor([6, 6])
('Hasekura', 'Echeverria') tensor([ 3, 10])
('Seah', 'Tommii') tensor([4, 3])
('Grushelevsky', 'Halin') tensor([6, 6])
('Hruska', 'Abboud') tensor([0, 2])
('Prevost', 'Alldritt') tensor([9, 9])
('Raffel', 'Sepulveda') tensor([ 0, 10])
('Abolins', 'Morjitsky') tensor([6, 6])
('Glazychev', 'Ahearn') tensor([6, 8])
('Havener', 'Sinha') tensor([1, 9])
('Malouf', 'Pavlenkov') tensor([2, 6])
('Adaksin', 'Auerbach') tensor([6, 6])
('Tsei', 'Grohovsky') tensor([6, 6])
('Anderson', 'Simmons') tensor([14,  9])
('Han

('Sabbagh', 'Adyrhaev') tensor([2, 6])
('Mcintosh', 'Cham') tensor([14,  2])
('Avladeev', 'Farren') tensor([6, 9])
('Tzelovalnikov', 'Ruhlin') tensor([6, 6])
('Nahas', 'Aldana') tensor([ 2, 10])
('Phung', 'Kemp') tensor([5, 9])
('Overson', 'Fell') tensor([9, 9])
('Jemlihanov', 'Maalouf') tensor([6, 2])
('Kalinichev', 'Huber') tensor([6, 1])
('Dzhemal', 'Haggett') tensor([6, 9])
('Aguzaroff', 'Grulich') tensor([6, 0])
('Munehin', 'Zeng') tensor([6, 4])
('Vyshegorodtsev', 'Awertcheff') tensor([6, 6])
('Shaidenko', 'Beatty') tensor([6, 9])
('Rutkowski', 'Belnikov') tensor([17,  6])
('Boulos', 'Chabrov') tensor([2, 6])
('Vaskin', 'Vanja') tensor([6, 6])
('Holzmann', 'Botros') tensor([1, 2])
('Ruadhain', 'Tuma') tensor([8, 2])
('Gribakin', 'Hoffmann') tensor([6, 1])
('Akhtar', 'Martsenyuk') tensor([9, 6])
('Yakupov', 'Gerges') tensor([6, 2])
('Koo', 'Poroh') tensor([4, 6])
('Muchalon', 'Black') tensor([0, 9])
('Nurullin', 'Zamorano') tensor([ 6, 10])
('Hairyuzov', 'Mohammed') tensor([6, 9])

('Naysmith', 'Shammas') tensor([9, 2])
('Agaev', 'Berezovoi') tensor([6, 6])
('Avtuhov', 'Matsumara') tensor([6, 3])
('Agababoff', 'Tomimoto') tensor([6, 3])
('Ustenko', 'Djemal') tensor([6, 6])
('Scetintsev', 'Dozorny') tensor([6, 6])
('Turik', 'Pinder') tensor([6, 9])
('Danisevich', 'Duggan') tensor([6, 9])
('Oddie', 'Collinson') tensor([9, 9])
('Zimmerman', 'Vandroogenbroeck') tensor([ 1, 15])
('Mintz', 'Mahlov') tensor([6, 6])
('Jopson', 'Lopes') tensor([9, 9])
('Yankelevich', 'Aldersley') tensor([6, 9])
('Williams', 'Qiao') tensor([9, 4])
('Koutsoubos', 'Kabachnik') tensor([11,  6])
('Limansky', 'Jigalev') tensor([6, 6])
('Verstakov', 'Notoriano') tensor([ 6, 12])
('Worgan', 'Jury') tensor([9, 9])
('Vaidanovich', 'Abbakumov') tensor([6, 6])
('Althuis', 'Guldin') tensor([15,  6])
('Severin', 'Oram') tensor([7, 9])
('Haslavsky', 'Samuel') tensor([6, 7])
('Patsiorkovsky', 'Saji') tensor([6, 3])
('Grojantsev', 'Timaev') tensor([6, 6])
('Bajukov', 'Corlett') tensor([6, 9])
('Greco', 'C

('Stoppelbein', 'Jilov') tensor([1, 6])
('Bosko', 'Albini') tensor([17, 12])
('Mochulov', 'Sinclair') tensor([6, 9])
('Gaber', 'Mifsud') tensor([2, 2])
('Worrall', 'Bateman') tensor([9, 9])
('Hajjar', 'Sexton') tensor([2, 9])
('Diduh', 'Vertinsky') tensor([6, 6])
('Larkin', 'Losa') tensor([ 9, 10])
('Grotus', 'Curley') tensor([6, 9])
('Harms', 'Bautista') tensor([ 6, 10])
('Janov', 'Amatore') tensor([ 6, 12])
('Bagishvili', 'Moshetov') tensor([6, 6])
('Franks', 'Golotik') tensor([9, 6])
('Lebedenko', 'Nishimoto') tensor([6, 3])
('Awdiyants', 'Roncalli') tensor([ 6, 12])
('Zhidomirov', 'Murchadh') tensor([6, 8])
('Makhovikov', 'Khoury') tensor([6, 2])
('Cutts', 'Didichenko') tensor([9, 6])
('Martinson', 'Antonowitsch') tensor([6, 0])
('Tsvelev', 'Hogan') tensor([6, 9])
("D'aramitz", 'Fourakis') tensor([ 7, 11])
('Gordeev', 'Hadad') tensor([6, 2])
('Wasem', 'Durakov') tensor([2, 6])
('Bazen', 'Kojima') tensor([6, 3])
('Valentinovitch', 'Joltovsky') tensor([6, 6])
('Ghanem', 'Baer') tenso

('Maroun', "Ren'Kas") tensor([2, 6])
('Tovar', 'Wyman') tensor([9, 1])
('Zhitkov', 'Good') tensor([6, 9])
('Whittle', 'Gou') tensor([9, 4])
('Edley', 'Tihodeev') tensor([9, 6])
('Noh', 'Elphick') tensor([16,  9])
('Chukhnov', 'Younge') tensor([6, 9])
('Ruvelas', 'Bagin') tensor([11,  6])
('Tchamov', 'Odonoghue') tensor([6, 9])
('Raikatuji', 'Edney') tensor([3, 9])
('Glavin', 'Chekhlakovsky') tensor([6, 6])
('Dorrian', 'Jeludev') tensor([9, 6])
('Zobkob', 'Kara') tensor([6, 0])
('Rahal', 'Rigin') tensor([2, 6])
('Salucci', 'Graifer') tensor([12,  6])
('Dioli', 'Mihnenko') tensor([12,  6])
('Kalikhov', 'Issa') tensor([6, 2])
('Ghanem', 'Golovatsky') tensor([2, 6])
('Minding', 'Juferov') tensor([6, 6])
('Jowett', 'Morgenstern') tensor([9, 1])
('Tupikhin', 'Shalenkov') tensor([6, 6])
('Shahmagon', 'Basara') tensor([6, 2])
('Ableuhov', 'Pendik') tensor([6, 6])
('Linsby', 'Valentinov') tensor([9, 6])
('Atiyeh', 'Stawski') tensor([ 2, 17])
('Antic', 'Gafiyatullin') tensor([9, 6])
('Ezakiya', 

# 1. Model

In [188]:
model = RNNClassifier()

# 2. Criterion & Loss
Loss is absorbed in the `test() and train()` already

In [189]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [192]:
for epoch in range(1, 1 + 1):
    train(model, 'cpu', train_loader, optimizer, epoch)
    test(model, 'cpu', test_loader)

AttributeError: 'tuple' object has no attribute 'to'